In [6]:
import sys
sys.path.append('../')
from torch.utils.data import Dataset
%reload_ext autoreload
%autoreload 2
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from mylib import get_motion, animate3d, get_SMPL_layer, t2m_bone, coco_bone, display_animate2D
import pickle
import torch
from tqdm import tqdm
import glob
import random
from math import cos, sin

In [49]:
smpl_data = pickle.load(open('/home/epinyoan/git/smplpytorch/smplpytorch/native/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl', 'rb'), encoding='latin1')
smpl_j_regressor = np.array(smpl_data['J_regressor'].toarray()).astype(np.float32)
coco_head_regressor = np.load('/home/epinyoan/git/Pose2Mesh_RELEASE/data/COCO/J_regressor_coco.npy').astype(np.float32)
coco_head_regressor = coco_head_regressor[:5]

rest_pose = torch.zeros((1,24,3))
verts, rest_joints = get_SMPL_layer(rest_pose.view(1,72), display=False)

coco_head_joints = torch.matmul(torch.from_numpy(coco_head_regressor), verts).numpy()
coco_head_joints.shape

torch.Size([24, 6890])


(1, 5, 3)

### Rotation functions

In [7]:
def normalize(x):
    return x/np.linalg.norm(x, axis=-1, keepdims=True)

def get_3d_angle(x, to_vector= np.array([0,0,1]), rotation_axis=np.array([0,1,0])):
    # need to check negative sign
    # https://stackoverflow.com/questions/5188561/signed-angle-between-two-3d-vectors-with-same-origin-within-the-same-plane
    # https://www.cuemath.com/geometry/angle-between-vectors/
    to_vector = normalize(to_vector)
    x = normalize(x)
    cosine = np.inner(x, to_vector)
    angle = np.arccos(cosine)
    
    # check negative angle by cross product
    c = np.cross(x,to_vector)
    sign = np.inner(c, rotation_axis) < 0
    np.negative(angle, where=sign, out=angle)
    return angle

def rotate2d_by_angle(vector, theta):
    rot = np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
    return np.dot(rot, vector)

### DIRO
http://www-labs.iro.umontreal.ca/~labimage/GaitDataset/ <br>
Skeleton detail: http://www.iro.umontreal.ca/~labimage/GaitDataset/dataset.pdf <br>
kinect joints: https://www.researchgate.net/publication/333524942_An_ontology_for_reasoning_on_body-based_gestures/figures?lo=1 <br>
9 subjects × 9 gaits × 1200 frames x 25 joints x 3D

In [74]:
data = np.load('/home/epinyoan/dataset/DIRO/DIRO_skeletons.npz', allow_pickle=True)
for k in data.files:
    print(k)
# data['data'] : 9x9x1200x75= (n_subject, n_gait, n_frame, 25*3
_data = np.reshape(data['data'], (9,9,1200, 25, 3))

data
split


In [30]:
DIRO_BONES = [(20,2), (2,3), (20,4), (4,5), (5,6), (20,8), (8,9), (9,10), (20,1), (1,0), (0,12), (12,13), (13,14), (0,16), (16,17), (17,18)]
def axis_standard(skeleton):
    skeleton = skeleton.copy()
#     skeleton[:, :, [0, 1, 2]] = skeleton[:, :, [2,0,1]]
    return skeleton
animate3d(_data[4,0,:300], DIRO_BONES, axis_standard=axis_standard)

In [83]:
def adjust_data(data):
    offset_rotation = .23
    # add head
    head = coco_head_joints.copy()
    head[0, :, [0,2]] = rotate2d_by_angle(coco_head_joints[0, :, [0,2]], np.pi)
    head[0, :, [1,2]] = rotate2d_by_angle(head[0, :, [1,2]], -2*offset_rotation)
    head = np.repeat(head, data.shape[0], axis=0)
    head += (data[:, [2]]+data[:, [3]])/2 - head[:, [0]]
    
    # rotate body up & move by feet
    data = np.concatenate((data, head), axis=1)
    for i in range(len(data)):
        data[i, :, [1,2]] = rotate2d_by_angle(data[i, :, [1,2]], offset_rotation)
        if i-1 >=0:
            diff_left = data[i, 14, 2] - data[i-1, 14, 2]
            diff_right = data[i, 18, 2] - data[i-1, 18, 2]
            if diff_left > diff_right:
                data[i, :, 2] -= diff_left
            else:
                data[i, :, 2] -= diff_right
    return data
__data = adjust_data(_data[5,0,600:660].copy())


DIRO_BONES = [(20,2), (2,3), (20,4), (4,5), (5,6), (20,8), (8,9), (9,10), (20,1), (1,0), (0,12), (12,13), (13,14), (0,16), (16,17), (17,18),
             # head
             [25, 27], [27, 29], [25, 26], [26, 28]]
animate3d(__data, DIRO_BONES)

In [135]:
# 25 nose      0
# 26 left eye  1
# 27 right eye 2
# 28 left ear  3
# 29 right ear 4

# 4 "lshoulder", # 5
# 8 "rshoulder", # 6
# 5 "lelbow",    # 7
# 9 "relbow",    # 8
# 6 "lwrist",    # 9
# 10 "rwrist",    # 10
# 12 "lhip",      # 11
# 16 "rhip",      # 12
# 13 "lknee",     # 13
# 17 "rknee",     # 14
# 14 "lankle",    # 15
# 18 "rankle",    # 16
coco_id = [25,26,27,28,29,4,8,5,9,6,10,12,16,13,17,14,18]
coco_data = adjust_data(_data[5,0,600:660].copy())[:, coco_id]
animate3d(coco_data[:60], coco_bone)

In [136]:
subjects = None
for subject in tqdm(_data):
    walks = None
    for walk in subject:
        d = adjust_data(walk.copy())
        d = d[:, coco_id]
        d = np.expand_dims(d, axis=0)
        if walks is None:
            walks = d
        else:
            walks = np.concatenate((walks,d))
    walks = np.expand_dims(walks, axis=0)
    if subjects is None:
        subjects = walks
    else:
        subjects = np.concatenate((subjects,walks))
np.save('/home/epinyoan/dataset/DIRO/DIRO3D.npy', subjects)
subjects.shape

100%|██████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  5.72it/s]


(9, 9, 1200, 17, 3)